In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("parkinson_10%.csv")

In [3]:
df

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,class
0,1.49,0.0,NaN,0.79,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,NaN,0.01,0.0,0.00,0.0,0.00,23.0,1
1,0.73,0.0,0.35,0.38,NaN,5.86,0.64,2.06,3.18,7.19,...,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0,1
2,1.22,0.0,0.73,0.67,NaN,8.72,0.88,4.35,5.17,NaN,...,211.44,1431.0,NaN,0.01,0.0,10.66,1.0,0.18,23.0,1
3,2.50,0.0,1.16,NaN,3.47,13.51,NaN,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0,1
4,NaN,0.0,1.72,NaN,5.15,9.11,1.04,3.10,4.93,12.82,...,NaN,117.0,114.0,0.00,0.0,18.18,NaN,13.32,23.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0
1036,1.76,0.0,NaN,0.69,2.43,NaN,0.74,2.72,3.93,NaN,...,143.80,47.0,44.0,0.01,0.0,NaN,NaN,25.79,1.0,0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,10.14,14.24,...,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0,0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,NaN,...,451.37,75.0,70.0,0.01,0.0,16.67,3.0,19.58,1.0,0


In [4]:
df.dtypes

Jitter (local)                         float64
Jitter (local, absolute)               float64
Jitter (rap)                           float64
Jitter (ppq5)                          float64
Jitter (ddp)                           float64
Shimmer (local)                        float64
Shimmer (local, dB)                    float64
Shimmer (apq3)                         float64
Shimmer (apq5)                         float64
Shimmer (apq11)                        float64
Shimmer (dda)                          float64
AC                                     float64
NTH                                    float64
HTN                                    float64
Median pitch                           float64
Mean pitch                             float64
Standard deviation                     float64
Minimum pitch                          float64
Maximum pitch                          float64
Number of pulses                       float64
Number of periods                      float64
Mean period  

In [5]:
df.isnull().sum()

Jitter (local)                         110
Jitter (local, absolute)               119
Jitter (rap)                           110
Jitter (ppq5)                          112
Jitter (ddp)                           112
Shimmer (local)                        106
Shimmer (local, dB)                    101
Shimmer (apq3)                         100
Shimmer (apq5)                          99
Shimmer (apq11)                         94
Shimmer (dda)                          106
AC                                     110
NTH                                    110
HTN                                    111
Median pitch                           119
Mean pitch                             112
Standard deviation                     103
Minimum pitch                          125
Maximum pitch                          119
Number of pulses                       102
Number of periods                      111
Mean period                            113
Standard deviation of period           100
Fraction of

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 29120
nilai hilang : 2912
persentase : 10.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Jitter (local)                         10.576923
Jitter (local, absolute)               11.442308
Jitter (rap)                           10.576923
Jitter (ppq5)                          10.769231
Jitter (ddp)                           10.769231
Shimmer (local)                        10.192308
Shimmer (local, dB)                     9.711538
Shimmer (apq3)                          9.615385
Shimmer (apq5)                          9.519231
Shimmer (apq11)                         9.038462
Shimmer (dda)                          10.192308
AC                                     10.576923
NTH                                    10.576923
HTN                                    10.673077
Median pitch                           11.442308
Mean pitch                             10.769231
Standard deviation                      9.903846
Minimum pitch                          12.019231
Maximum pitch                          11.442308
Number of pulses                        9.807692
Number of periods   

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        x1 = np.nan_to_num(x1)  # Mengganti nilai NaN dengan 0
        x2 = np.nan_to_num(x2)  # Mengganti nilai NaN dengan 0
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",dist)

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,7)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
    
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [11]:
data = df.iloc[:, :-1]
df2 = df.copy()
dummy = df2.fillna(0)
dummy.fillna(0, inplace=True)
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,class
0,1.49,0.0,0.00,0.79,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,0.0,0.01,0.0,0.00,0.0,0.00,23.0,1
1,0.73,0.0,0.35,0.38,0.00,5.86,0.64,2.06,3.18,7.19,...,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0,1
2,1.22,0.0,0.73,0.67,0.00,8.72,0.88,4.35,5.17,0.00,...,211.44,1431.0,0.0,0.01,0.0,10.66,1.0,0.18,23.0,1
3,2.50,0.0,1.16,0.00,3.47,13.51,0.00,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0,1
4,0.00,0.0,1.72,0.00,5.15,9.11,1.04,3.10,4.93,12.82,...,0.00,117.0,114.0,0.00,0.0,18.18,0.0,13.32,23.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0
1036,1.76,0.0,0.00,0.69,2.43,0.00,0.74,2.72,3.93,0.00,...,143.80,47.0,44.0,0.01,0.0,0.00,0.0,25.79,1.0,0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,10.14,14.24,...,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0,0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,0.00,...,451.37,75.0,70.0,0.01,0.0,16.67,3.0,19.58,1.0,0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 6442


In [13]:
euc_dist = {}

In [14]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,7)

Kolom : 1
[0.0, 0.89, 2.16, 2.33, 2.57, 2.68, 4.34]
min :  0.0
max :  4.34
mean :  2.138571428571429
fuzzy :  [(0.0, 0.0), (2.68, 0.754055807916937), (2.16, 0.9902660609993511), (4.34, 0.0), (2.57, 0.8040233614536016), (0.89, 0.41616566466265864), (2.33, 0.9130434782608696)]
fuzzy :  None
pembilang :  8.72396304180934
penyebut :  3.877554373293418
rata-rata tertimbang :  2.249862207451034
 
[1.29, 1.53, 2.11, 2.29, 2.5, 2.72, 3.05]
min :  1.29
max :  3.05
mean :  2.212857142857143
fuzzy :  [(2.11, 0.8885448916408665), (2.29, 0.907849829351536), (1.29, 0.0), (2.5, 0.6569965870307168), (2.72, 0.3941979522184298), (3.05, 0.0), (1.53, 0.2600619195046439)]
fuzzy :  None
pembilang :  7.066410465030272
penyebut :  3.107651179746193
rata-rata tertimbang :  2.273875044626919
 
[0.0, 0.41, 1.16, 1.24, 1.63, 2.36, 2.83]
min :  0.0
max :  2.83
mean :  1.375714285714286
fuzzy :  [(0.0, 0.0), (1.63, 0.8251473477406681), (2.83, 0.0), (1.16, 0.8431983385254411), (0.41, 0.29802699896157836), (1.24, 0.9

[0.0, 0.0, 1.22, 1.3, 1.86, 2.87, 3.83]
min :  0.0
max :  3.83
mean :  1.582857142857143
fuzzy :  [(3.83, 0.0), (1.22, 0.7707581227436823), (0.0, 0.0), (2.87, 0.4272091544818817), (0.0, 0.0), (1.3, 0.8212996389891697), (1.86, 0.8766687857596948)]
fuzzy :  None
pembilang :  4.864708655309245
penyebut :  2.8959357019744285
rata-rata tertimbang :  1.6798400088760677
 
[1.22, 1.48, 1.6798400088760677, 1.86, 2.3, 2.87, 3.83]
min :  1.22
max :  3.83
mean :  2.1771200012680096
fuzzy :  [(3.83, 0.0), (2.87, 0.5808044145591122), (1.6798400088760677, 0.4804413326091435), (1.86, 0.6686726838349598), (2.3, 0.9256570357035854), (1.22, 0.0), (1.48, 0.27164827780795237)]
fuzzy :  None
pembilang :  6.248755067426267
penyebut :  2.9272237445147535
rata-rata tertimbang :  2.134703600684998
 
[0.0, 0.0, 1.63, 2.31, 2.69, 2.92, 3.04]
min :  0.0
max :  3.04
mean :  1.7985714285714285
fuzzy :  [(0.0, 0.0), (2.69, 0.28193325661680096), (2.31, 0.5880322209436133), (3.04, 0.0), (0.0, 0.0), (2.92, 0.09666283084

[0.0, 0.0, 1.01, 1.91, 2.344332992086947, 2.92, 4.91]
min :  0.0
max :  4.91
mean :  1.8706189988695638
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (1.01, 0.5399282272928664), (2.92, 0.6547385797502386), (1.91, 0.9870430850506109), (2.344332992086947, 0.8441412929010234), (4.91, 0.0)]
fuzzy :  None
pembilang :  6.321364737813958
penyebut :  3.0258511849947394
rata-rata tertimbang :  2.089119507648539
 
[0.0, 0.0, 1.63, 1.697295996322187, 2.2, 2.83, 2.95]
min :  0.0
max :  2.95
mean :  1.615327999474598
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (2.2, 0.5619358162190843), (2.83, 0.08990973059505357), (1.63, 0.9890070365455885), (1.697295996322187, 0.9385856623834756), (2.95, 0.0)]
fuzzy :  None
pembilang :  4.6958424898041775
penyebut :  2.5794382457432024
rata-rata tertimbang :  1.8204903713254763
 
[0.0, 2.833707256679253, 3.2, 3.44, 3.46, 3.62, 4.24]
min :  0.0
max :  4.24
mean :  2.9705296080970363
fuzzy :  [(0.0, 0.0), (3.46, 0.614429454184247), (4.24, 0.0), (3.44, 0.6301840555735867), (3.62, 0.48839

[0.0, 2.0696416456288627, 2.2844988636977823, 2.31, 2.69, 2.84, 3.04]
min :  0.0
max :  3.04
mean :  2.176305787046664
fuzzy :  [(0.0, 0.0), (3.04, 0.0), (2.69, 0.40523601380076635), (2.2844988636977823, 0.874732196848743), (2.0696416456288627, 0.9509884401113738), (2.84, 0.2315634364575809), (2.31, 0.8452065430701695)]
fuzzy :  None
pembilang :  7.66668214106263
penyebut :  3.3077266302886335
rata-rata tertimbang :  2.3178100846845475
 
[0.0, 0.47, 0.84, 1.49, 1.78, 2.03, 2.41]
min :  0.0
max :  2.41
mean :  1.2885714285714285
fuzzy :  [(0.0, 0.0), (1.78, 0.5617834394904458), (2.03, 0.3388535031847136), (1.49, 0.8203821656050955), (0.47, 0.3647450110864745), (2.41, 0.0), (0.84, 0.6518847006651884)]
fuzzy :  None
pembilang :  3.6292298642789556
penyebut :  2.737648820031918
rata-rata tertimbang :  1.3256740008883399
 
[0.0, 1.98, 2.16, 2.33, 2.44, 2.57, 3.01]
min :  0.0
max :  3.01
mean :  2.07
fuzzy :  [(0.0, 0.0), (2.33, 0.7234042553191486), (2.16, 0.9042553191489359), (2.44, 0.60638

[0.0, 3.31, 3.5959388283788227, 3.95, 4.06, 4.18, 5.23]
min :  0.0
max :  5.23
mean :  3.4751341183398314
fuzzy :  [(0.0, 0.0), (5.23, 0.0), (4.06, 0.6667176176980185), (3.95, 0.7294004706439857), (4.18, 0.5983363235751449), (3.5959388283788227, 0.9311601465949606), (3.31, 0.952481224402723)]
fuzzy :  None
pembilang :  14.590158998794553
penyebut :  3.8780957829148326
rata-rata tertimbang :  3.7621966592656926
 
[0.0, 2.19, 3.2563840306440595, 5.34, 5.58, 5.89, 5.99]
min :  0.0
max :  5.99
mean :  4.035197718663437
fuzzy :  [(0.0, 0.0), (5.89, 0.05115606880283935), (5.34, 0.3325144472184542), (5.99, 0.0), (5.58, 0.2097398820916403), (3.2563840306440595, 0.806994912686127), (2.19, 0.5427243353828484)]
fuzzy :  None
pembilang :  7.063736576437161
penyebut :  1.9431296461819092
rata-rata tertimbang :  3.6352368923591
 
[0.0, 1.38, 2.82, 3.0929727165415706, 3.96, 4.32, 5.33]
min :  0.0
max :  5.33
mean :  2.9861389595059387
fuzzy :  [(0.0, 0.0), (1.38, 0.4621352250225901), (3.0929727165415

...

[0, 0, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.7142857142857143
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (0, 1), (1, 1)]
fuzzy :  1
pembilang :  5
penyebut :  7
rata-rata tertimbang :  1
 
[0, 0, 0, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.5714285714285714
fuzzy :  [(0, 1), (0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (0, 1)]
fuzzy :  1
pembilang :  4
penyebut :  7
rata-rata tertimbang :  1
 
[0, 0, 0, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.5714285714285714
fuzzy :  [(0, 1), (0, 1), (1, 1), (1, 1), (0, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  4
penyebut :  7
rata-rata tertimbang :  1
 
[0, 0, 0, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.5714285714285714
fuzzy :  [(0, 1), (0, 1), (1, 1), (1, 1), (1, 1), (0, 1), (1, 1)]
fuzzy :  1
pembilang :  4
penyebut :  7
rata-rata tertimbang :  1
 
[0, 0, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.7142857142857143
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (0, 1)]
fuz

[0, 0, 0, 0, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.42857142857142855
fuzzy :  [(0, 0), (0, 0), (1, 0), (0, 0), (1, 0), (0, 0), (1, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 0, 0, 1, 1]
median :  4
min :  0
max :  1
mean :  0.2857142857142857
fuzzy :  [(0, 0), (1, 0), (0, 0), (0, 0), (0, 0), (0, 0), (1, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 0, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.5714285714285714
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (0, 1), (0, 1), (1, 1)]
fuzzy :  1
pembilang :  4
penyebut :  7
rata-rata tertimbang :  1
 
[0, 0, 0, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.5714285714285714
fuzzy :  [(0, 1), (1, 1), (0, 1), (1, 1), (1, 1), (0, 1), (1, 1)]
fuzzy :  1
pembilang :  4
penyebut :  7
rata-rata tertimbang :  1
 
[0, 0, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.7142857142857143
fuzzy :  [(0, 1), (1, 1), (0, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fu

[0, 1, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.8571428571428571
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  6
penyebut :  7
rata-rata tertimbang :  1
 
[0, 0, 0, 0, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.42857142857142855
fuzzy :  [(0, 0), (1, 0), (0, 0), (0, 0), (0, 0), (1, 0), (1, 0)]
fuzzy :  0
pembilang :  0
penyebut :  0
rata-rata tertimbang :  0
 
[0, 0, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.7142857142857143
fuzzy :  [(0, 1), (1, 1), (0, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  5
penyebut :  7
rata-rata tertimbang :  1
 
[0, 0, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.7142857142857143
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (0, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  5
penyebut :  7
rata-rata tertimbang :  1
 
[0, 1, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.8571428571428571
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fu

[0, 1, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.8571428571428571
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  6
penyebut :  7
rata-rata tertimbang :  1
 
[0, 1, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.8571428571428571
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  6
penyebut :  7
rata-rata tertimbang :  1
 
[0, 1, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.8571428571428571
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  6
penyebut :  7
rata-rata tertimbang :  1
 
[0, 0, 0, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.5714285714285714
fuzzy :  [(0, 1), (0, 1), (0, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  4
penyebut :  7
rata-rata tertimbang :  1
 
[0, 0, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.7142857142857143
fuzzy :  [(0, 1), (0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuz

[0, 0, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.7142857142857143
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (0, 1)]
fuzzy :  1
pembilang :  5
penyebut :  7
rata-rata tertimbang :  1
 
[0, 1, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.8571428571428571
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  6
penyebut :  7
rata-rata tertimbang :  1
 
[0, 1, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.8571428571428571
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  6
penyebut :  7
rata-rata tertimbang :  1
 
[0, 1, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.8571428571428571
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuzzy :  1
pembilang :  6
penyebut :  7
rata-rata tertimbang :  1
 
[0, 1, 1, 1, 1, 1, 1]
median :  4
min :  0
max :  1
mean :  0.8571428571428571
fuzzy :  [(0, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
fuz

In [15]:
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,class
0,1.490000,0.0,0.946218,0.790000,2.700000,8.330000,0.780000,4.52,4.610000,6.800000,...,187.580000,160.0,176.451072,0.01,0.0,4.517237,1.0,3.390000,23.0,1
1,0.730000,0.0,0.350000,0.380000,1.611904,5.860000,0.640000,2.06,3.180000,7.190000,...,234.510000,170.0,169.000000,0.01,0.0,2.250000,1.0,27.699201,23.0,1
2,1.220000,0.0,0.730000,0.670000,0.772408,8.720000,0.880000,4.35,5.170000,5.380976,...,211.440000,1431.0,1044.419564,0.01,0.0,10.660000,1.0,0.180000,23.0,1
3,2.500000,0.0,1.160000,1.248625,3.470000,13.510000,0.870426,5.26,8.770000,16.780000,...,220.230000,94.0,92.000000,0.01,0.0,12.589198,1.0,0.180000,23.0,1
4,2.249862,0.0,1.720000,1.186573,5.150000,9.110000,1.040000,3.10,4.930000,12.820000,...,236.707546,117.0,114.000000,0.01,0.0,18.180000,1.0,13.320000,23.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.130000,0.0,2.720000,2.510000,8.160000,9.850000,0.970000,4.59,6.810000,8.300000,...,158.590000,63.0,60.000000,0.01,0.0,24.190000,2.0,15.110000,1.0,1
1036,1.760000,0.0,1.011427,0.690000,2.430000,7.289524,0.740000,2.72,3.930000,7.162062,...,143.800000,47.0,44.000000,0.01,0.0,17.700370,1.0,25.790000,1.0,1
1037,2.910000,0.0,1.140000,1.270000,3.420000,15.490000,1.460000,6.43,10.140000,14.240000,...,149.640000,79.0,77.000000,0.01,0.0,20.000000,1.0,4.570000,1.0,1
1038,3.790000,0.0,1.790000,1.480000,5.360000,15.700000,1.370000,6.37,11.090000,10.605580,...,451.370000,75.0,70.000000,0.01,0.0,16.670000,3.0,19.580000,1.0,1


In [16]:
dummy.describe()

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,class
count,1040.000000,1040.0,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,...,1040.000000,1040.000000,1040.000000,1.040000e+03,1040.00000,1040.000000,1040.000000,1040.000000,1040.000000,1040.0
mean,2.637785,0.0,1.219395,1.324106,3.646706,12.727828,1.174554,5.583714,7.854242,12.017481,...,231.934061,108.524438,104.217209,1.000000e-02,0.00001,28.354330,1.587919,16.656759,12.742620,1.0
std,1.709237,0.0,0.918497,1.103837,2.806184,5.313069,0.405598,2.871439,4.716101,5.832423,...,119.300323,149.617978,142.499411,1.700847e-16,0.00031,19.267141,1.336304,12.729838,15.387512,0.0
min,0.190000,0.0,0.060000,0.080000,0.190000,1.190000,0.100000,0.500000,0.710000,0.520000,...,85.540000,6.000000,4.000000,1.000000e-02,0.00000,0.160000,1.000000,0.180000,1.000000,1.0
25%,1.530000,0.0,0.640000,0.684115,1.908416,9.385000,0.930000,3.700000,5.237504,8.227500,...,142.790000,42.000000,40.000000,1.000000e-02,0.00000,12.277703,1.000000,6.936071,1.000000,1.0
50%,2.350000,0.0,1.010000,1.100000,3.040000,12.080000,1.155316,5.050000,6.915000,11.260000,...,194.560000,64.000000,61.000000,1.000000e-02,0.00000,26.210000,1.000000,13.194292,5.000000,1.0
75%,3.330130,0.0,1.560000,1.630000,4.655000,14.917500,1.380000,6.760000,9.220000,14.810000,...,260.382500,111.000000,108.000000,1.000000e-02,0.00000,41.704766,1.699870,23.440000,23.000000,1.0
max,14.380000,0.0,8.020000,13.540000,24.050000,41.140000,2.720000,25.820000,72.860000,44.760000,...,526.410000,1490.000000,1483.000000,1.000000e-02,0.01000,88.160000,12.000000,69.120000,55.000000,1.0


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
x = dummy.values
y = df.iloc[:, -1].values

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=47)

In [26]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [27]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[131   5]
 [ 10 114]]
0.9423076923076923
